In [1]:
import torch as pt
import torchaudio as ta
from torch.utils.data import Dataset,DataLoader

import numpy as numpy

import os
import subprocess
import tqdm as tqdm
import json

import librosa as lb

F:\anacondapython\envs\tf\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [21]:
path='data\public_dataset'
for file in os.listdir(path):
    if file.endswith('webm'):
        name,ext=os.path.splitext(file)
        new_name=name+'.wav'
        os.rename(os.path.join(path,file),os.path.join(path,new_name))

In [13]:
def convert_webm_to_wav(file):
    command = ['ffmpeg', '-i', file, file[:-5] + '.wav']
    subprocess.run(command)

In [19]:
path='data\public_dataset'
for file in os.listdir(path):
    if file.endswith('webm'):
        subprocess.run(['ffmpeg','-i',os.path.join(path,file),'test_df/'+file[:-5]+'.wav'])

In [2]:
path='test_df'
name_set=set()
for file in os.listdir(path):
    if file.endswith('wav'):
        name_set.add(file)
print(len(name_set))

25984


In [24]:
t=os.path.join(path,list(name_set)[0])
label_path='data\public_dataset'
fname=t[8:-4]
l=os.path.join(label_path,fname+'.json')
print(fname)
with open(l,'r') as f:
    content=json.loads(f.read())
print(content)

2d71f678-6687-4a2f-bfed-e40fd23b9752
{'datetime': '2020-04-14T18:46:15.579555+00:00', 'cough_detected': '0.8779', 'latitude': '37.8', 'longitude': '29.1', 'age': '8', 'gender': 'male', 'respiratory_condition': 'False', 'fever_muscle_pain': 'False', 'status': 'healthy'}


In [27]:
print(list(name_set)[0])

2d71f678-6687-4a2f-bfed-e40fd23b9752.wav


In [6]:
signal,sr=ta.load(t)

In [16]:
print(content['cough_detected'])

0.9902


In [34]:
class CoughVidDataset(Dataset):

    def __init__(self,audio_path,label_path):
        name_set=set()
        for file in os.listdir(audio_path):
            if file.endswith('wav'):
                name_set.add(file)
        name_set=list(name_set)
        self.datalist=name_set
        self.audio_path=audio_path
        self.label_path=label_path
        
    def __len__(self):
        return len(self.datalist)

    def __getitem__(self,idx):
        audio_file_path=os.path.join(self.audio_path,self.datalist[idx])
        label_file_path=os.path.join(self.label_path,self.datalist[idx][:-4]+'.json')
        waveform,sample_rate=ta.load(audio_file_path)
        with open(label_file_path,'r') as f:
            content=json.loads(f.read())
            f.close()
        label=content['cough_detected']
        return waveform,label


In [35]:
audio_path='test_df'
label_path='data\public_dataset'
coughvid_dataset=CoughVidDataset(audio_path,label_path)